In [38]:
import pandas as pd
from parsing.requirements_profile.extract_requirements import extract_requirement
from sentence_transformers import SentenceTransformer, util
import numpy
import torch
from transformers import AutoTokenizer, AutoModel
import os

In [ ]:
import spacy
import de_core_news_lg

In [2]:
!pip uninstall numpy -y

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2


In [5]:
!pip install numpy==1.24.1

In [37]:
!pip install pandas
!pip install torch
!pip install transformers
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached scikit_learn-1.5.2-cp310-cp310-macosx_10_9_x86_64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp310-cp310-macosx_10_13_x86_64.whl.metadata (60 kB)
  Using cached pillow-11.0.0-cp310-cp310-macosx_10_10_x86_64.whl.metadata (9.1 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached pillow-11.0.0-cp310-cp310-macosx_10_10_x86_64.whl (3.2 MB)
Using cached scikit_learn-1.5.2-cp310-cp310-macosx_10_9_x86_64.whl (12.1 MB)
Using cached scipy-1.14.1-cp310-cp310-macosx_10_13_x86_64.whl (39.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [ ]:
!pip install spacy
!python -m spacy download de_core_news_lg

In [9]:
esco_df = pd.read_csv("./skills_de.csv")
esco_df

,conceptType,conceptUri,skillType,reuseLevel,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,scopeNote,definition,inScheme,description
0,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0005c151-5b5a...,skill/competence,sector-specific,Musikpersonal verwalten,NaN,NaN,released,2023-11-30T15:53:37.136Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Zuweisen und Verwalten der Aufgaben des Person...
1,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00064735-8fad...,skill/competence,occupation-specific,Strafvollzugsverfahren beaufsichtigen,NaN,NaN,released,2023-11-30T15:04:00.689Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Überwachen des Betriebs einer Justizvollzugsan...
2,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/000709ed-2be5...,skill/competence,sector-specific,nicht unterdrückende Praktiken anwenden,NaN,NaN,released,2023-11-28T10:45:53.54Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Ermitteln von Repressionen in Gesellschaften, ..."
3,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0007bdc2-dd15...,skill/competence,sector-specific,Einhaltung von Vorschriften von Eisenbahnfahrz...,NaN,NaN,released,2023-11-30T16:29:18.273Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Kontrollieren von Fahrzeugen, Komponenten und ..."
4,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00090cc1-1f27...,skill/competence,cross-sector,verfügbare Dienste ermitteln,NaN,NaN,released,2023-11-28T10:38:49.206Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Ermitteln der verschiedenen verfügbaren Dienst...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13934,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/ffef5eb3-a15e...,skill/competence,sector-specific,berufliche Leistungsfähigkeit von Nutzern/Nutz...,NaN,NaN,released,2024-02-09T17:29:15.235Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,"Wiederherstellen der kognitiven, sensomotorisc..."
13935,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff0b074-5a76...,skill/competence,sector-specific,Beleuchtung in Transportgeräten einbauen,Beleuchtung in Transportmittel installieren,NaN,released,2024-02-09T17:29:14.494Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Einbau von Beleuchtungselementen in Transportg...
13936,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff0e2cd-d0bd...,knowledge,sector-specific,Verarbeitung natürlicher Sprache,Verarbeitung natürlicher Sprachen\nNatural Lan...,NaN,released,2024-02-09T17:29:15.235Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Technologien, die es IKT-Geräten ermöglichen, ..."
13937,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff5bc45-b506...,skill/competence,cross-sector,Bauarbeiten koordinieren,NaN,NaN,released,2024-02-09T17:29:13.7Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Koordinierung der Tätigkeiten mehrerer Bauarbe...


In [14]:
filename = ("./TRESCON/Anforderungsprofile/AKP_Constantia_Head of HR_11329.docx")
requirements = (extract_requirement(filename))
test_req = requirements[1]
test_list = test_req.split("\n")
cleaned_list = [item for item in test_list if item != ""]
cleaned_list = cleaned_list[1:]
cleaned_list

['Führung und Weiterentwicklung des Bereiches Personal, inkl. neuer Methoden und Innovationen im HR-Bereich',
 'Weiterentwicklung des HR-Teams (aktuell 3 MA) – fachlich & persönlich',
 'Ansprechpartner für die Führungskräfte bei allen Personalfragen',
 'Hauptansprechpartner für Betriebsräte in allen personellen und sozialen Angelegenheiten, v. a. im Zusammenhang mit Betriebsvereinbarungen und Sonderfällen',
 'Lösung von arbeitsrechtlichen und sozialrechtlichen Problemstellungen',
 'Hauptinitiator für Employer-Branding (intern & extern) und Unternehmenskultur in enger Zusammenarbeit mit Marketing',
 'Weiterentwicklung des Recruiting-Konzepts',
 'Gestaltung der Personalentwicklung; Leitung ausgewählter Personalentwicklungsprojekte wie z.B.: Leadership-Programme oder Talente-Programm',
 'Aufgaben rund um das Personalsystem: Arbeitszeit, Entlohnungssysteme (u.a. anstehendes Projekt Zeiterfassung), Personalkennzahlen, Personalplanung, Personalkostenmanagement, Berichtswesen, KPIs, etc.',
 '

In [3]:
nlp_de = de_core_news_lg.load()

In [5]:
def get_n_grams(tokens, n=2, use_lemmas=False):
    result= []
    for idx, token in enumerate(tokens):
        check = True
        for i in range(idx, idx+n):
            check = check and i < len(tokens)
            if not check:
                continue
            check = check and (tokens[i].pos_ == "NOUN" or tokens[i].pos_ == "ADJ" or tokens[i].pos_ == "VERB")    
        if check:
            if use_lemmas:
                result.append(" ".join([t.lemma_ for t in tokens[idx:idx+n]]))
            else:
                result.append(" ".join([str(t) for t in tokens[idx:idx+n]]))
    return "; ".join([ngram for ngram in result])

In [17]:
keyword_list = []
for text in cleaned_list:
    if '\t' in text:
        text = text.replace('\t', '')
        print(f"TAB  {text}")
    else:
        print(text)
    tokens = nlp_de(text)
    x = get_n_grams(tokens, n=2)
    print(x)
    keyword_list.append(x)
    print("----")

Führung und Weiterentwicklung des Bereiches Personal, inkl. neuer Methoden und Innovationen im HR-Bereich
Bereiches Personal; neuer Methoden
----
Weiterentwicklung des HR-Teams (aktuell 3 MA) – fachlich & persönlich

----
Ansprechpartner für die Führungskräfte bei allen Personalfragen

----
Hauptansprechpartner für Betriebsräte in allen personellen und sozialen Angelegenheiten, v. a. im Zusammenhang mit Betriebsvereinbarungen und Sonderfällen
sozialen Angelegenheiten
----
Lösung von arbeitsrechtlichen und sozialrechtlichen Problemstellungen
sozialrechtlichen Problemstellungen
----
Hauptinitiator für Employer-Branding (intern & extern) und Unternehmenskultur in enger Zusammenarbeit mit Marketing
enger Zusammenarbeit
----
Weiterentwicklung des Recruiting-Konzepts

----
Gestaltung der Personalentwicklung; Leitung ausgewählter Personalentwicklungsprojekte wie z.B.: Leadership-Programme oder Talente-Programm
Leitung ausgewählter; ausgewählter Personalentwicklungsprojekte
----
Aufgaben rund 

In [16]:
model_name = "google-bert/bert-base-german-cased"
tokenizer_ger = AutoTokenizer.from_pretrained(model_name)
model_ger = AutoModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ger = model_ger.to(device)

In [31]:
# for bert-base-german-cased
def compute_sentence_embedding(texts, model, tokenizer):
    encoded_inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    input_ids = encoded_inputs["input_ids"].to(device)
    attention_mask = encoded_inputs["attention_mask"].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    # extract CLS-Token-Embeddings
    cls_embeddings = outputs.last_hidden_state[:, 0, :]
    return cls_embeddings

In [40]:
def compute_and_save_embeddings(esco_terms, model, tokenizer, file_path="esco_embeddings.npy"):
    if os.path.exists(file_path):
        print("load esco embeddings")
        esco_embeddings = np.load(file_path)
    else:
        print("calculate and save embeddings")
        esco_embeddings = compute_sentence_embedding(esco_terms, model, tokenizer)
        np.save(file_path, esco_embeddings) 
    return esco_embeddings

esco_embeddings = compute_and_save_embeddings(esco_df['description'].tolist(), model_ger, tokenizer_ger)

load esco embeddings


In [ ]:
def get_embeddings(texts, model, tokenizer, device):
    encoded_inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    input_ids = encoded_inputs["input_ids"].to(device)
    attention_mask = encoded_inputs["attention_mask"].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    # extract CLS-Token-Embeddings
    cls_embeddings = outputs.last_hidden_state[:, 0, :]
    return cls_embeddings

In [35]:
fp = "esco_embeddings.npy"
if os.path.exists(fp):
    print("load esco embeddings")
    esco_embeddings = np.load(fp)
else:
    print("calculate and save embeddings")
    esco_embeddings = get_embeddings(esco_df['description'][:4000].tolist(), model_ger, tokenizer_ger, device)
    np.save(fp, esco_embeddings) 

load esco embeddings


In [39]:
# embeddings for requirements
anforderungs_embedding = get_embeddings(cleaned_list, model_ger, tokenizer_ger, device)

# embeddings for esco
#esco_embeddings = np.load("esco_embeddings.npy")

similarity_matrix = util.pytorch_cos_sim(anforderungs_embedding, esco_embeddings)

print("Best Matching ESCO Term for Each Requirement:")
for i, term in enumerate(cleaned_list):
    similarities = similarity_matrix[i].cpu()
    best_match_idx = similarities.argmax()
    best_match_term = esco_df['description'].iloc[best_match_idx.item()]
    best_score = similarities[best_match_idx.item()]

    print(f"Anforderungsprofil-Begriff: {term}")
    print(f"   Bester ESCO-Begriff: {best_match_term} - Ähnlichkeitsscore = {best_score:.2f}")

manhattan_distances = torch.cdist(anforderungs_embedding, torch.from_numpy(esco_embeddings), p=1)
print("--------------------------------------------------------------------------------------")
print("Best Matching ESCO Term for Each Requirement (using Manhattan Distance):")
for i, term in enumerate(cleaned_list):
    distances = manhattan_distances[i].cpu().numpy()  
    best_match_idx = distances.argmin()  
    best_match_term = esco_df['description'].iloc[best_match_idx.item()] 
    best_distance = distances[best_match_idx]

    print(f"Anforderungsprofil-Begriff: {term}")
    print(f"   Bester ESCO-Begriff: {best_match_term} - Manhattan-Distanz = {best_distance:.2f}")

Best Matching ESCO Term for Each Requirement:
Anforderungsprofil-Begriff: Führung und Weiterentwicklung des Bereiches Personal, inkl. neuer Methoden und Innovationen im HR-Bereich
   Bester ESCO-Begriff: Analyse und Bestimmung des Informationsbedarfs junger Menschen und Anpassung der Dienste und des Ansatzes an ihre individuellen oder kollektiven Bedürfnisse.
    - Ähnlichkeitsscore = 0.84
Anforderungsprofil-Begriff: Weiterentwicklung des HR-Teams (aktuell 3 MA) – fachlich & persönlich
   Bester ESCO-Begriff: Teilnehmen an Aktivitäten zur Förderung der Qualität, insbesondere im Hinblick auf den Erwerb und die Bewertung von Ausrüstung, Ressourcen, sicherem Speicher- und Lieferantenmanagement. - Ähnlichkeitsscore = 0.84
Anforderungsprofil-Begriff: Ansprechpartner für die Führungskräfte bei allen Personalfragen
   Bester ESCO-Begriff: Beraten der Kunden in Bezug auf die Zubereitung von Fleisch und Fleischwaren. - Ähnlichkeitsscore = 0.81
Anforderungsprofil-Begriff: Hauptansprechpartner fü